In [2]:
import h5py
import numpy as np
import pandas as pd
import json
import yaml
from ludwig import LudwigModel
import copy
import ray
from ludwig.utils.misc import merge_dict
import re

In [3]:
pattern = "^{{.*}}$"
list_index_pattern = "^\[\d+\]$"


def get_keys_from_list(l, path=""):
    parameters = []

    for index, v in enumerate(l):
        if isinstance(l[index], str):
             if re.match(pattern, l[index], flags=0) is not None:
                parameters.append([path + "[" + index + "]", v])

        elif isinstance(l[index], dict):
            p = get_keys_from_dict(l[index], path+ "[" + str(index) + "]" +"->" )
            if p:
                for l in p:
                    parameters.append(l)
        elif isinstance(l[index], list):
            p = get_keys_from_list(l[index], path+ "[" + str(index) + "]" +"->" )
            if p:
                for l in p:
                    parameters.append(l)

    return parameters


def get_keys_from_dict(dct, path=""):
    parameters = []
    for k, v in dct.items():
        if isinstance(dct[k], str):
            if re.match(pattern, dct[k], flags=0) is not None:
                parameters.append([path + k,dct[k]])

        elif isinstance(dct[k], dict):
            p = get_keys_from_dict(dct[k], path+k+"->" )
            if p:
                for l in p:
                    parameters.append(l)

        elif isinstance(dct[k], list):
            p = get_keys_from_list(dct[k], path+k+"->" )
            if p:
                for l in p:
                    parameters.append(l)

    return parameters



def update_param(dct, path, value):
    if len(path) == 1:
        dct[path[0]] = value
    else:
        l = re.match(list_index_pattern, path[0], flags=0)
        if l is not None and isinstance(dct, list):
            index = int(l.group(0)[1:-1])
            update_param(dct[index], path[1:], value)
        else:
             update_param(dct[path[0]], path[1:], value)


def build_model(base_model, config):
    for k, v in config.items():
        p = k.split('->')
        update_param(base_model, p, v)
    return base_model

In [8]:
from ray.tune import grid_search

def build_search_space(annotated_model):
    pattern = "^{{(.*)}}$"
    grid_search_space = {}
    keys = get_keys_from_dict(annotated_model)
    for a in keys:
        grid_search_space[a[0]]= grid_search(yaml.load(re.match(pattern, a[1], flags=0)[1]))

    return grid_search_space


In [11]:
#use full path for Ray
data = '/Users/bm255022/Projects/Teradata/ludwig/examples/hyperparameters/titanic.hdf5'
metadata = '/Users/bm255022/Projects/Teradata/ludwig/examples/hyperparameters/titanic.json'

def train(base, config, reporter):
    new_model_def = build_model(base, config)
    model = LudwigModel(new_model_def)
    train_stats = model.train(data_hdf5=data, train_set_metadata_json=metadata)
    return reporter(mean_accuracy=train_stats['validation']['Survived']['accuracy'][-1], done=True)


In [ ]:
from ray.tune import register_trainable, grid_search, run_experiments

ray.shutdown()
ray.init()

with open("titanic.yaml", 'r') as stream:
    annotated_model = yaml.load(stream)

grid_search_space = build_search_space(annotated_model)
register_trainable('train', lambda cfg, rptr: train(annotated_model, cfg, rptr))
run_experiments({'my_experiment': {
    'run': 'train',
    'stop': {'mean_accuracy': 0.9},
    'config': grid_search_space}
    })
